```python
task = threading.Thread(target=_func, args=(args,) )
task.start()
task.join()

name = threading.current_thread().getName()
task.setName("name")
```

## 1. 线程使用

In [23]:
import threading

loop = 1000000
num = 0

def _add(loop):
    global num

    for i in range(loop): 
        num += 1

task = threading.Thread(target=_add, args=(loop,) )
task.start() 

task.join() # 主线程等待子线程结束并回收
print(num) # 主线程打印，不会等待子线程结束


1000000


In [ ]:
# set / get name
import threading

def task():
    name = threading.current_thread().getName()
    print(name)

for i in range(10):
    td = threading.Thread(target=task)
    td.setName(f"name-{i}") # must before start()
    td.start()

In [53]:
# Thread Class：另一个使用线程的方法
class MyThread(threading.Thread):
    def run(self):  # t.start() 后会默认执行run()
        print(sum(self._args))  # Thread.__init__ 完成了 self._args的init

all_lists=([1,2,3,4], [1,4,56,6])

for lst in all_lists:
    t = MyThread(args=lst)
    t.start()   

## 2. Thread Lock and Safety  
线程安全类数据（操作时自带锁）：
1. list： append; extend; x = L[i]; pop; L[3:5]; sort(); List1 = List2; x.filed = y  
2. dict: D[x]=y; D1.update(D2); D.keys

In [16]:
import threading

lock = threading.RLock()
num = 0
loop = 100000

def add_task(count):
    global num

    lock.acquire()          # 写法1
    for i in range(count):
        num += 1
    lock.release()

def sub_task(count):
    global num

    with lock:             # 写法2
        for i in range(count):
            num -= 1


t1 = threading.Thread(target=add_task, args=(loop,))
t2 = threading.Thread(target=sub_task, args=(loop,))
t1.start()
t2.start()

t1.join()
t2.join()
num

0

## DeadLock  


In [ ]:
# 递归锁
lock = threading.RLock()

lock.acquire() 
lock.acquire() # 连续量程获取锁不会卡住

# Lock
lock = threading.Lock()

lock.acquire() 
lock.acquire() # 第二次获取，死锁在该位置


## Thread Pool  
```python
pool = ThreadPoolExecutor(100) # 创建线程池
pool.submit(fun, arg1, arg2) # 传入任务和参数
pool.shutdown(True) # 等待所有任务执行完
add_done_callback(function_name)
```

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import random

pool = ThreadPoolExecutor(100) # 创建线程池
pool.submit(fun, arg1, arg2) # 传入任务

pool.shutdown(True) # 等待所有任务执行完

# add_done_callback(function_name)
future = pool.submit(fun, arg1, arg2) 
future.add_done_callback(fn) # 子线程在future 完成的那一刻自动执行 fn；fn的参数为fun的return的掩码，用


In [ ]:
# future.result() 拿到该线程执行完后的函数
from concurrent.futures import ThreadPoolExecutor
import random

def task():
    return random.randint(0,10)

pool = ThreadPoolExecutor(10)

future_list = []

for _ in range(15):
    future = pool.submit(task)
    future_list.append(future)

pool.shutdown()
for fu in future_list:
    print(fu.result())